In [ ]:
# 1. 구글 드라이브 마운트 (데이터 로드용)
from google.colab import drive
drive.mount('/content/drive')

# 2. 필수 라이브러리 설치 (A100은 빠르니까 금방 됩니다)
!pip install -q gliner kiwipiepy bm25s networkx scipy sentence-transformers llama-index llama-index-embeddings-huggingface llama-index-llms-huggingface python-mecab-ko faiss-cpu llama-index-vector-stores-faiss

In [ ]:
# 코랩에서 사용하시면 아래 주석 해제해 주세요
# import os
# # 🛑 [중요] 욕심쟁이 프레임워크들의 메모리 점유 방지 (Import 전에 실행 필수!)
# os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"       # TensorFlow가 필요할 때만 메모리 쓰게 함
# os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"  # JAX가 메모리 미리 찜하는 거 금지
# os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".10"   # 혹시 쓰더라도 10%만 쓰게 제한

from datasets import load_from_disk
from huggingface_hub import login
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from typing import List, Dict, Callable, Optional
import numpy as np
import json
import tqdm
from functools import partial

# LlamaIndex 관련
from llama_index.core import Document, VectorStoreIndex, Settings, StorageContext
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.retrievers import QueryFusionRetriever, BaseRetriever
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.schema import TextNode, NodeWithScore, QueryBundle, BaseNode
from llama_index.vector_stores.faiss import FaissVectorStore
import faiss

# 기타 라이브러리
import bm25s
from gliner import GLiNER
from sentence_transformers import CrossEncoder

# Hugging Face 로그인
print("🚀 환경 설정 시작!")
HF_TOKEN = ""
os.environ["HUGGINGFACE_HUB_TOKEN"] = HF_TOKEN
login(token=HF_TOKEN)

print(f"GPU 사용 가능 여부: {torch.cuda.is_available()}")

In [ ]:
# wiki data load
with open('/data/wikipedia_documents.json') as f:
    wiki_data = json.load(f)
id_to_title = {v["document_id"]: v["title"] for v in wiki_data.values()}


train_set_dir = "/data/train_dataset/"
dataset = load_from_disk(train_set_dir)

In [ ]:
# --- 모델 로드 설정 ---
GEMMA_MODEL_NAME = "google/gemma-3-4b-it"  # 메모리 효율성을 위해 4b 대신 9b를 예시로 사용 (사용자 환경에 따라 변경 가능)
EMBEDDING_MODEL_NAME = "BAAI/bge-m3"
RERANKER_MODEL_NAME = "BAAI/bge-reranker-v2-m3"

In [ ]:
# --- LLM 및 Tokenizer 로드 ---
def load_gemma():
    """Gemma 모델과 토크나이저를 로드합니다."""
    # Q: Gemma 3-4b-it 사용 예정이었는데, 현재는 Gemma 2-9b-it을 사용하려 합니다.
    # A: VRAM 상황에 따라 모델 이름을 적절히 변경하여 사용하세요.
    tokenizer = AutoTokenizer.from_pretrained(GEMMA_MODEL_NAME)
    model = AutoModelForCausalLM.from_pretrained(
        GEMMA_MODEL_NAME,
        device_map="cuda" if torch.cuda.is_available() else "cpu",
        torch_dtype=torch.bfloat16,
    )
    return tokenizer, model

In [ ]:
documents: List[Document] = []
for doc_id, data in wiki_data.items():
    # 'text' 필드를 문서 내용으로 사용
    documents.append(
        Document(
            text=data['text'],
            metadata={
                "document_id": data['document_id'],
                "title": data['title'],
                "corpus_source": data['corpus_source']
            }
        )
    )

In [ ]:
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
import json

# 3. 문서 청킹 (Node 생성)
# SentenceSplitter는 문장 단위 분할을 기본으로 하면서,
# 최종 청크 크기를 chunk_size=512로 제한합니다.
splitter = SentenceSplitter(chunk_size=256, chunk_overlap=128)

# nodes에는 작은 텍스트 청크(TextNode)들이 리스트 형태로 담깁니다.
nodes: List[TextNode] = splitter.get_nodes_from_documents(documents)

print(f"원본 문서 개수: {len(documents)}개")
print(f"생성된 청크(Node) 개수: {len(nodes)}개")
print(f"첫 번째 청크 텍스트 예시: {nodes[0].get_content()[:100]}...")


# 🚨 [중요] 이번엔 까먹지 말고 바로 저장합시다!
save_path = "/my_search_index"
os.makedirs(save_path, exist_ok=True)

with open(f"{save_path}/nodes.pkl", "wb") as f:
    pickle.dump(nodes, f)

print("💾 Nodes 파일 영구 저장 완료! (이제 꺼져도 안전함)")

In [ ]:
import pickle

# 🚨 [중요] 이번엔 까먹지 말고 바로 저장합시다!
save_path = "my_search_index"
os.makedirs(save_path, exist_ok=True)

with open(f"{save_path}/nodes.pkl", "wb") as f:
    pickle.dump(nodes, f)

print("💾 Nodes 파일 영구 저장 완료! (이제 꺼져도 안전함)")

In [ ]:
import pickle
import os
from google.colab import drive


# 2. 파일 경로 설정
save_path = "my_search_index"
node_file_path = os.path.join(save_path, "nodes.pkl")

# 3. 불러오기 (Load)
if os.path.exists(node_file_path):
    print(f"📂 '{node_file_path}' 파일을 로드 중입니다...")

    with open(node_file_path, "rb") as f:
        nodes = pickle.load(f)

    print(f"✅ 로드 완료! 총 {len(nodes)}개의 청크(Node)가 복구되었습니다.")

    # 4. 잘 불러와졌는지 눈으로 확인 (첫 번째 노드 내용)
    # (에러 없이 내용이 출력되면 성공입니다!)
    try:
        print(f"👀 첫 번째 노드 내용: {nodes[0].text[:50]}...")
    except:
        print("👀 내용 확인 실패 (형식이 다를 수 있음)")

else:
    print(f"😱 파일이 없습니다! 경로를 다시 확인해주세요: {node_file_path}")

In [ ]:
embed_model = HuggingFaceEmbedding(
    model_name=EMBEDDING_MODEL_NAME,
    device="cuda" if torch.cuda.is_available() else "cpu",
    embed_batch_size = 512
)

In [ ]:
# faiss vs

dummy_emb = embed_model.get_text_embedding("dim 체크용")
dim = len(dummy_emb)
faiss_index = faiss.IndexFlatIP(dim)
vector_store = FaissVectorStore(faiss_index=faiss_index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [ ]:
vector_index = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
    embed_model=embed_model,
    show_progress=True
)

In [ ]:
# 구글 드라이브 경로
vector_save_path = "vector_store_index"

# 저장 실행
vector_index.storage_context.persist(persist_dir=vector_save_path)

print(f"✅ 벡터 인덱스 영구 저장 완료: {vector_save_path}")

In [ ]:
# from llama_index.core import StorageContext, load_index_from_storage
# from llama_index.embeddings.huggingface import HuggingFaceEmbedding
# import os

# # 1. 저장된 경로 설정 (아까 저장한 그 경로)
# vector_save_path = "/vector_store_index"

# # 2. 임베딩 모델 준비 (필수! 🚨)
# # 저장할 때 BAAI/bge-m3 모델로 숫자를 만들었으니, 불러올 때도 똑같은 번역기가 필요합니다.
# embed_model = HuggingFaceEmbedding(
#     model_name="BAAI/bge-m3",
#     device="cuda",
# )

# # 3. 인덱스 로드 (Load)
# if os.path.exists(vector_save_path):
#     print(f"📂 '{vector_save_path}'에서 벡터 인덱스를 불러옵니다...")

#     # 저장소 컨텍스트(데이터 창고) 재구성
#     storage_context = StorageContext.from_defaults(persist_dir=vector_save_path)

#     # 인덱스 객체로 부활시키기
#     vector_index = load_index_from_storage(
#         storage_context,
#         embed_model=embed_model # 모델 주입
#     )

#     print("🎉 벡터 인덱스 복구 완료! (다시 인덱싱할 필요 없음)")

# else:
#     print(f"😱 경로에 파일이 없습니다! 확인해주세요: {vector_save_path}")

# # 4. (선택) 바로 검색기로 만들어서 쓸 수 있습니다
# # dense_retriever = vector_index.as_retriever(similarity_top_k=100)

In [ ]:
# --- 3. Reranker (사용자 정의) ---
from sentence_transformers import CrossEncoder

class Reranker:
    def __init__(self, model_name: str = RERANKER_MODEL_NAME):
        self.model = CrossEncoder(model_name, device="cuda" if torch.cuda.is_available() else "cpu")

    def rerank(self, query: str, docs: List[str], doc_ids: List[str], top_k: int = 5):
        """
        [수정됨] 중복된 doc_id는 제거하고 가장 점수 높은 대표 1개만 남깁니다.
        """
        if not docs:
            return [], []

        # 1. 점수 계산
        pairs = [[query, d] for d in docs]
        scores = self.model.predict(pairs)

        # 2. 하나로 묶기 (점수, 텍스트, 문서ID)
        # 3개를 한 번에 zip으로 묶어서 관리하는 게 편합니다.
        combined = list(zip(scores, docs, doc_ids))

        # 3. 점수 높은 순으로 정렬
        combined.sort(key=lambda x: x[0], reverse=True)

        # 4. [핵심] 중복 제거 로직 (Deduplication) 🚨
        seen_ids = set()
        final_results = []
        final_ids = []

        for score, text, did in combined:
            # 이미 등록된 문서 ID라면 스킵! (가장 점수 높은 놈만 이미 들어갔을 테니까)
            if did in seen_ids:
                continue

            seen_ids.add(did)
            final_results.append((text, score)) # 결과 포맷 유지
            final_ids.append(did)

            # top_k개 채웠으면 그만
            if len(final_results) >= top_k:
                break

        return final_results, final_ids

In [ ]:
reranker = Reranker()

In [ ]:
from kiwipiepy import Kiwi
import re
from typing import List

def _fallback_tokenize(text: str) -> list[str]:
    """Kiwi 실패 시 단순 whitespace + 문자 기반 토큰화"""
    # 공백 분리 + 알파벳/숫자/기타 유니코드 단어 추출
    tokens = re.findall(r'\b\w+\b', text, re.UNICODE)
    return [t for t in tokens]

def tokenize_kiwi(
    text: str,
    kiwi: Kiwi,
    tag_include: List[str],
    text_type: str,
    top_n: int,
    score_threshold: float = 1.2,
) -> list[str]:
    try:
        # 1. 토큰화할 텍스트가 문서(Corpus)일 때
        if text_type == "corpus":
            # 문서는 길 수 있으므로 top_n을 유동적으로 설정
            analyzed = kiwi.analyze(text, top_n=top_n + len(text) // 200)

            if not analyzed:
                return _fallback_tokenize(text)

            num_candi = 1
            # 1위 점수 기준 threshold 이내의 후보군 추가
            while (
                num_candi < len(analyzed)
                and analyzed[num_candi][1] > score_threshold * analyzed[0][1]
            ):
                num_candi += 1

        # 2. 토큰화할 텍스트가 쿼리(Query)일 때
        elif text_type == "query":
            analyzed = kiwi.analyze(text, top_n=top_n)

            if not analyzed:
                return _fallback_tokenize(text)

            num_candi = 3 # 쿼리는 좀 더 다양하게 후보를 봄

        # 3. 후보군에서 토큰 추출
        all_tokenized = [
            (t.form, t.tag)
            for nc in range(num_candi)
            for t in analyzed[nc][0]
        ]

        # 4. 중복 제거
        unique_tokenized = set(all_tokenized)

        # 5. [핵심 수정] 필터링은 하되, 태그(/NNG 등)는 떼고 '단어'만 리스트에 담음
        filtered = [
            form  # 🚨 수정됨: f"{form}/{tag}" -> form
            for form, tag in unique_tokenized
            if tag in tag_include
        ]

        return filtered if filtered else _fallback_tokenize(text)

    except Exception:
        return _fallback_tokenize(text)

In [ ]:
kiwi_tags = ['NNG', 'NNP', 'NNB', 'NR', 'VV', 'VA', 'MM', 'XR', 'SW', 'SL', 'SH', 'SN', 'SB']

GLINER_TAG = {
    "언어": {"CV_LANGUAGE": "문명_언어", "TR_HUMANITIES": "이론_철학/언어/역사"}, "문학": {"FD_HUMANITIES": "학문 분야_인문학", "AFA_DOCUMENT": "인공물_도서/서적 작품명"}, "역사": {"CV_CULTURE": "문명_문명/문화", "AF_CULTURAL_ASSET": "인공물_문화재", "DT_DYNASTY": "날짜_왕조시대", "DT_GEOAGE": "날짜_지질시대", "EV_WAR_REVOLUTION": "사건_전쟁/혁명", "EV_OTHERS": "사건_기타"}, "철학": {"TR_HUMANITIES": "이론_철학/언어/역사", "CV_TRIBE": "문명_민족/종족"}, "교육": {"OGG_EDUCATION": "기관_교육", "OGG_LIBRARY": "기관_도서관"}, "민속": {"CV_CULTURE": "문명_문명/문화", "EV_FESTIVAL": "사건_축제/영화제"}, "인문 일반": {"FD_HUMANITIES": "학문 분야_인문학"},
    "법률": {"CV_LAW": "문명_법/법률", "OGG_LAW": "기관_법률", "CV_POLICY": "문명_제도/정책"}, "군사": {"OGG_MILITARY": "기관_군사", "AF_WEAPON": "인공물_무기"}, "경영": {"OGG_ECONOMY": "기관_경제", "AFW_SERVICE_PRODUCTS": "인공물_서비스 상품"}, "경제": {"OGG_ECONOMY": "기관_경제", "CV_CURRENCY": "문명_통화", "CV_TAX": "문명_조세"}, "복지": {"CV_FUNDS": "문명_연금/기금"}, "정치": {"OGG_POLITICS": "기관_정부/공공", "CV_POLICY": "문명_제도/정책", "EV_ACTIVITY": "사건_사회운동/선언"}, "매체": {"OGG_MEDIA": "기관_미디어", "AFA_VIDEO": "인공물_영화/TV 프로그램"}, "행정": {"OGG_POLITICS": "기관_정부/공공"}, "심리": {"FD_SOCIAL_SCIENCE": "학문 분야_사회과학"}, "사회 일반": {"FD_SOCIAL_SCIENCE": "학문 분야_사회과학"},
    "지구": {"FD_SCIENCE": "학문 분야_과학", "MT_ROCK": "물질_암석"}, "지리": {"LC_OTHERS": "장소_기타", "LCG_MOUNTAIN": "장소_산/산맥", "LCG_RIVER": "장소_강/호수", "LCG_OCEAN": "장소_바다", "LCG_ISLAND": "장소_섬", "LCG_CONTINENT": "장소_대륙", "TM_DIRECTION": "용어_방향"}, "해양": {"LCG_OCEAN": "장소_바다", "LCG_BAY": "장소_반도/만"}, "천문": {"LC_SPACE": "장소_천체", "FD_SCIENCE": "학문 분야_과학"}, "환경": {"TM_CLIMATE": "용어_기후 지역", "FD_SCIENCE": "학문 분야_과학"}, "생명": {"TM_CELL_TISSUE_ORGAN": "용어_세포/조직/기관", "FD_SCIENCE": "학문 분야_과학"}, "동물": {"AM_INSECT": "동물_곤충", "AM_BIRD": "동물_조류", "AM_FISH": "동물_어류", "AM_MAMMALIA": "동물_포유류", "AM_AMPHIBIA": "동물_양서류", "AM_REPTILIA": "동물_파충류", "AM_TYPE": "동물_분류명", "AM_PART": "동물_부위명", "AM_OTHERS": "동물_기타"}, "식물": {"PT_FLOWER": "식물_꽃", "PT_GRASS": "식물_풀", "PT_TYPE": "식물_분류명", "PT_PART": "식물_부위명", "PT_OTHERS": "식물_기타", "PT_TREE": "식물_나무", "PT_FRUIT": "식물_과일/열매"}, "천연자원": {"MT_ELEMENT": "물질_원소"}, "수학": {"FD_SCIENCE": "학문 분야_과학", "TM_SHAPE": "용어_모양/형태", "QT_SIZE": "수량_넓이/면적", "QT_LENGTH": "수량_길이/거리", "QT_VOLUME": "수량_부피", "QT_PERCENTAGE": "수량_백분율"}, "물리": {"FD_SCIENCE": "학문 분야_과학", "TR_SCIENCE": "이론_과학", "QT_SPEED": "수량_속도", "QT_TEMPERATURE": "수량_온도", "QT_WEIGHT": "수량_무게"}, "화학": {"MT_CHEMICAL": "물질_화학", "MT_ELEMENT": "물질_원소", "MT_METAL": "물질_금속"}, "자연 일반": {"FD_SCIENCE": "학문 분야_과학"},
    "농업": {"PT_FRUIT": "식물_과일/열매", "FD_OTHERS": "학문 분야_기타"}, "수산업": {"AM_FISH": "동물_어류"}, "임업": {"PT_TREE": "식물_나무"}, "광업": {"MT_ROCK": "물질_암석", "MT_METAL": "물질_금속"}, "공업": {"AFW_OTHER_PRODUCTS": "인공물_기타 상품"}, "서비스업": {"AFW_SERVICE_PRODUCTS": "인공물_서비스 상품", "OGG_HOTEL": "기관_숙박 업체"}, "산업 일반": {"OGG_ECONOMY": "기관_경제"},
    "의학": {"FD_MEDICINE": "학문 분야_의학", "TR_MEDICINE": "이론_의학", "TMM_DISEASE": "용어_증상/질병"}, "약학": {"TMM_DRUG": "용어_약품"}, "한의": {"FD_MEDICINE": "학문 분야_의학"}, "수의": {"FD_MEDICINE": "학문 분야_의학"}, "식품": {"CV_FOOD": "문명_음식", "CV_DRINK": "문명_음료/술", "CV_FOOD_STYLE": "문명_음식 유형"}, "보건 일반": {"OGG_MEDICINE": "기관_의료"},
    "건설": {"AF_BUILDING": "인공물_건축물/토목건설물", "CV_BUILDING_TYPE": "문명_건축 양식"}, "교통": {"AF_TRANSPORT": "인공물_교통수단/운송수단", "AF_ROAD": "인공물_도로/철로"}, "기계": {"TMI_HW": "용어_IT 하드웨어"}, "전기·전자": {"TMI_HW": "용어_IT 하드웨어"}, "재료": {"MT_ELEMENT": "물질_원소"}, "정보·통신": {"TMI_SW": "용어_IT 소프트웨어", "TMI_HW": "용어_IT 하드웨어", "TMI_SITE": "용어_URL 주소", "TMI_EMAIL": "용어_이메일 주소", "TMI_MODEL": "용어_제품 모델명", "TMI_SERVICE": "용어_IT 서비스", "TMI_PROJECT": "용어_프로젝트"}, "공학 일반": {"FD_SCIENCE": "학문 분야_과학"},
    "체육": {"CV_SPORTS": "문명_스포츠", "OGG_SPORTS": "기관_스포츠", "CV_SPORTS_POSITION": "문명_스포츠 포지션", "CV_SPORTS_INST": "문명_스포츠 용품/도구", "EV_SPORTS": "사건_스포츠 행사", "TM_SPORTS": "용어_스포츠"}, "연기": {"AFA_PERFORMANCE": "인공물_춤/공연/연극 작품명"}, "영상": {"AFA_VIDEO": "인공물_영화/TV 프로그램"}, "무용": {"AFA_PERFORMANCE": "인공물_춤/공연/연극 작품명"}, "음악": {"AFA_MUSIC": "인공물_음악 작품명", "AF_MUSICAL_INSTRUMENT": "인공물_악기", "OGG_ART": "기관_예술"}, "미술": {"AFA_ART_CRAFT": "인공물_미술/조형 작품명", "FD_ART": "학문 분야_예술", "TM_COLOR": "용어_색깔"}, "복식": {"CV_CLOTHING": "문명_의복/섬유"}, "공예": {"AFA_ART_CRAFT": "인공물_미술/조형 작품명"}, "예체능 일반": {"FD_ART": "학문 분야_예술"},
    "가톨릭": {"OGG_RELIGION": "기관_종교"}, "기독교": {"OGG_RELIGION": "기관_종교"}, "불교": {"OGG_RELIGION": "기관_종교"}, "종교 일반": {"OGG_RELIGION": "기관_종교"},
    "인명": {"PS_NAME": "인물_사람", "PS_CHARACTER": "인물_가상 캐릭터", "CV_OCCUPATION": "문명_직업", "CV_POSITION": "문명_직위/직책", "CV_RELATION": "문명_가족/친족 관계"}, "지명": {"LCP_COUNTRY": "장소_국가", "LCP_PROVINCE": "장소_도/주 지역", "LCP_COUNTY": "장소_세부 행정구역", "LCP_CITY": "장소_도시", "LCP_CAPITALCITY": "장소_수도", "LC_OTHERS": "장소_기타"}, "책명": {"AFA_DOCUMENT": "인공물_도서/서적 작품명"}, "고유명 일반": {"OGG_OTHERS": "기관_기타"}
}

gliner_labels = []

for middle_cat, inner_map in GLINER_TAG.items():
    for gliner_code, leaf_label in inner_map.items():
        gliner_labels.append(leaf_label)
gliner_labels = sorted(list(set(gliner_labels)))

KIWI_TO_URIMALSAEM_MAP = {
    "NNG": "명사", "NNP": "명사", "NNB": "의존 명사", "NR": "수사", "XR": "명사", "SN": "수사",
    "VV": "동사", "VA": "형용사", "MM": "관형사",
    "SW": None, "SB": None, "SL": None, "SH": None
}

In [ ]:
from kiwipiepy import Kiwi
from gliner import GLiNER
from typing import List, Dict, Set, Any

class RichHybridTokenizer:
    def __init__(self, gliner_model: GLiNER, labels: List[str], kiwi_tags: List[str]):
        print("🔧 Rich Hybrid Tokenizer (Raw Mode: 필터링 없음) 초기화...")
        self.gliner = gliner_model
        self.labels = labels
        self.kiwi = Kiwi()

        # 사용자가 요청한 태그 리스트를 그대로 사용
        self.target_tags = set(kiwi_tags)

        # 🚨 C++의 '++' 등을 잡기 위해 SW(기호)가 target_tags에 없다면 강제로 추가 권장
        # (사용자님이 전달주실 kiwi_tags에 SW가 포함되어 있다고 가정하거나, 여기서 추가)
        # 붙임표도 일단 가져옴 (필요 없으면 나중에 매핑에서 None 처리)

    def tokenize(self, text: str) -> List[Dict[str, Any]]:
        if not text: return []

        token_info = {}

        # -------------------------------------------------------
        # 1. GLiNer: 핵심 개체명 & 카테고리 추출
        # -------------------------------------------------------
        try:
            preds = self.gliner.predict_entities(
                text, self.labels, flat_ner=True, threshold=0.1
            )
            for e in preds:
                raw_token = e['text'] # 공백 유지 (예: Visual Basic)
                category = e['label']

                if raw_token not in token_info:
                    token_info[raw_token] = {
                        'text': raw_token,
                        'category': category,
                        'pos': '(-)'
                    }
                else:
                    token_info[raw_token]['category'] = category

        except Exception as e:
            print(f"⚠️ GLiNer Error: {e}")

        # -------------------------------------------------------
        # 2. Kiwi: 형태소 분석 & 품사 태깅
        # -------------------------------------------------------
        try:
            res = self.kiwi.analyze(text, top_n=1)
            if res:
                for token in res[0][0]:
                    # 1. 타겟 태그인지 확인
                    if token.tag in self.target_tags:
                        word = token.form

                        # 🚨 [삭제됨] SW, SO 반복 문자 필터링 로직 제거!
                        # 이제 "++", "C#", "~~~~" 모두 있는 그대로 들어옵니다.

                        # 저장 로직
                        if word in token_info:
                            # GLiNer와 겹치면 POS 정보 업데이트
                            token_info[word]['pos'] = token.tag
                        else:
                            # Kiwi만 찾은 단어 추가
                            token_info[word] = {
                                'text': word,
                                'category': None,
                                'pos': token.tag
                            }
        except Exception as e:
            print(f"⚠️ Kiwi Error: {e}")

        return list(token_info.values())

In [ ]:
from typing import List, Dict, Any
from collections import defaultdict
import json

# ==============================================================================
# 1. Helper Functions (사용자 제공 코드 그대로 사용)
# ==============================================================================

def create_leaf_to_middle_map(simplified_map: Dict[str, Dict[str, str]]) -> Dict[str, List[str]]:
    """분류 체계를 평탄화하여 (상세 분류 -> 중분류 리스트) 맵을 생성"""
    leaf_to_middle = defaultdict(set)
    # GLINER_TAG 구조: {중분류: {코드: 상세분류}}
    for middle_cat, tag_dict in simplified_map.items():
        for gliner_code, leaf_value in tag_dict.items():
            leaf_to_middle[leaf_value].add(middle_cat)
    return {k: sorted(list(v)) for k, v in leaf_to_middle.items()}

def transform_tokens_enrich_data(token_list: List[Dict[str, Any]], leaf_to_mid_map: Dict[str, List[str]]) -> List[Dict[str, Any]]:
    """
    입력 토큰 리스트에 'dict_cat'(중분류)과 'dict_pos'(사전 품사)를 추가합니다.
    """
    enriched_list = []

    # 전역 변수 KIWI_TO_URIMALSAEM_MAP 사용 (윗 셀에서 정의됨)
    # 혹시 모르니 안전장치로 get 사용
    global KIWI_TO_URIMALSAEM_MAP

    for token in token_list:
        raw_text = token.get('text')
        leaf_cat = token.get('category')
        raw_pos = token.get('pos')

        # 1. 중분류 조회 (GLINER_TAG 기반 역추적)
        middle_categories = leaf_to_mid_map.get(leaf_cat, [])

        # 2. 품사 매핑 (KIWI_TO_URIMALSAEM_MAP 사용)
        dict_pos = KIWI_TO_URIMALSAEM_MAP.get(raw_pos)

        new_token = {
            'text': raw_text,
            'category': leaf_cat,
            'pos': raw_pos,
            'dict_cat': middle_categories, # 예: ['인명']
            'dict_pos': dict_pos           # 예: '명사'
        }
        enriched_list.append(new_token)

    return enriched_list



In [ ]:
import pickle
import networkx as nx

# ==============================================================================
# 1. 지식 그래프 로드 함수
# ==============================================================================
def load_knowledge_graph(file_path: str) -> nx.Graph:
    """pkl 파일에서 NetworkX 그래프를 로드합니다."""
    try:
        with open(file_path, 'rb') as f:
            graph = pickle.load(f)
        print(f"✅ 그래프 로드 성공! (노드: {graph.number_of_nodes()}개, 엣지: {graph.number_of_edges()}개)")
        return graph
    except Exception as e:
        print(f"❌ 그래프 로드 실패: {e}")
        return None

class GraphRetriever:
    def __init__(self, graph: nx.Graph, min_weight: float = 0.7):
        print("🔍 GraphRetriever 초기화 (Word Indexing)...")
        self.graph = graph
        self.min_weight = min_weight
        self.word_index = defaultdict(list)
        self._build_word_index()

    def _build_word_index(self):
        for node_id, data in self.graph.nodes(data=True):
            word = data.get('word')
            if word:
                self.word_index[word].append(node_id)

    def retrieve(self, enriched_tokens: List[Dict[str, Any]]) -> Dict[str, float]:
        expanded_weights = defaultdict(float)
        original_texts = set()

        for item in enriched_tokens:
            raw_text = item['text']
            dict_cats = item['dict_cat']
            dict_pos = item['dict_pos']
            raw_pos = item['pos']

            # 1. 검색어 정규화
            search_text = raw_text
            if raw_pos in ['VV', 'VA']:
                search_text += '다'

            original_texts.add(raw_text)
            original_texts.add(search_text)

            # 2. 색인 조회
            candidate_ids = self.word_index.get(search_text, [])
            valid_ids = []

            # 3. 정밀 필터링 (수정된 로직) 🚨
            for nid in candidate_ids:
                node_cat = self.graph.nodes[nid].get('category')

                # Case A: GLiNER가 찾아준 중분류가 있으면 -> 강력 필터링 (유지)
                if dict_cats:
                    if node_cat in dict_cats:
                        valid_ids.append(nid)

                # Case B: 품사 정보만 있는 경우
                elif dict_pos:
                    # [수정 포인트] 동사/형용사는 엄격하게 검사
                    if dict_pos in ['동사', '형용사']:
                        if node_cat == dict_pos:
                            valid_ids.append(nid)

                    # [수정 포인트] 명사(NNG/NNP) 등은 카테고리 불일치 허용!
                    # "국가/명사" -> "국가/정치" (OK!)
                    else:
                        valid_ids.append(nid)

            # 4. 유의어 확장 (기존 동일)
            for nid in valid_ids:
                expanded_weights[search_text] = 1.0
                for neighbor, edge in self.graph[nid].items():
                    w = edge.get('weight', 0.0)
                    if w >= self.min_weight:
                        neighbor_word = self.graph.nodes[neighbor].get('word', neighbor)
                        if neighbor_word not in original_texts:
                            expanded_weights[neighbor_word] = max(expanded_weights[neighbor_word], w)

        return dict(expanded_weights)

In [ ]:
# 파일 경로 (아까 언급하신 경로로 가정)
KG_FILE_PATH = "/urimalsaem_graph_FINAL2.pkl"

# 1. 그래프 로드
synonym_graph = load_knowledge_graph(KG_FILE_PATH)

In [ ]:
from functools import partial
from typing import List

# -----------------------------------------------------------
# 1. Enrich 함수 이름 매칭 (중요!)
# -----------------------------------------------------------
# 아까 만든 긴 이름의 함수를 'enrich_tokens'라는 이름으로 쓰겠다고 선언
# (만약 위에서 def transform_tokens_enrich_data... 로 정의했다면)
if 'transform_tokens_enrich_data' in locals():
    enrich_tokens = transform_tokens_enrich_data
elif 'enrich_tokens' not in locals():
    print("⚠️ 경고: enrich_tokens 함수가 정의되지 않았습니다! 위쪽 셀을 실행해주세요.")

from typing import List, Dict, Any

def tokenize_query_rich(
    text: str,
    tokenizer_obj,
    l2m_map_obj,
    graph_retriever_obj
) -> Dict[str,float]:
    """
    [질문 분석] -> [풍부화] -> [그래프 확장] -> [BoW 병합(안전장치)] -> [단어 리스트 반환]
    """

    # 1. Rich Tokenizing (GLiNER + Kiwi)
    # 예: [{'text': '에케르트', ...}, {'text': '시행착오', ...}]
    tokens = tokenizer_obj.tokenize(text)

    # 2. Enrich (메타데이터 추가)
    enriched = enrich_tokens(tokens, l2m_map_obj)

    # 3. Graph Expansion (유의어 찾기)
    # 예: {'시행착오': 1.0, '시오법': 0.7}
    retrieval_results = graph_retriever_obj.retrieve(enriched)

    # ---------------------------------------------------------
    # 4. [사용자 로직 통합] Weighted BoW 생성 (Safety Net) 🚨
    # ---------------------------------------------------------
    final_bow = retrieval_results.copy()

    for token in enriched:
        raw_text = token['text']
        # 원본 단어가 유의어 사전에 없더라도 검색어에 무조건 포함시킴 (가중치 1.0)
        if raw_text not in final_bow:
            final_bow[raw_text] = 1.0

    # 5. 리스트로 변환 (BM25s 입력용)
    # 현재 KiwiBM25Retriever는 List[str] 입력을 받으므로 키값만 추출해서 넘깁니다.
    # (가중치 0.7 정보를 점수 계산에 직접 반영하려면 Retriever 내부 로직을 바꿔야 하지만,
    #  일단 '검색어에 포함시키는 것(Recall)'만으로도 효과는 충분합니다.)
    return final_bow

print("✅ 질문용 토크나이저 업데이트 완료 (Safety Net 적용됨)")

gliner_model = GLiNER.from_pretrained("lots-o/gliner-bi-ko-xlarge-v1")
gliner_model = gliner_model.to("cuda")

tokenizer = RichHybridTokenizer(
    gliner_model=gliner_model,  # 로드해둔 모델
    labels=gliner_labels,       # 라벨 리스트
    kiwi_tags=kiwi_tags         # 품사 리스트
)
l2m_map = create_leaf_to_middle_map(GLINER_TAG)
graph_retriever = GraphRetriever(synonym_graph, min_weight=0.7)

from functools import partial

query_tokenizer_rich = partial(
    tokenize_query_rich,
    tokenizer_obj=tokenizer,         # 로봇
    l2m_map_obj=l2m_map,             # 지도
    graph_retriever_obj=graph_retriever # 검색기
)

print("✅ 최종 조립 완료!")
print("✅ 질문용 Rich 토크나이저 조립 완료!")

In [ ]:
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.schema import NodeWithScore, QueryBundle, BaseNode
import bm25s
from collections import defaultdict
import tqdm

class KiwiWeightedBM25Retriever(BaseRetriever):
    """
    [통합 버전]
    1. 문서는 Kiwi로 빠르게 인덱싱
    2. 질문은 RichTokenizer가 준 가중치(Dict)를 받아
    3. 내부에서 점수를 곱하고 더해서(Weight Sum) 결과를 반환
    """
    def __init__(
        self,
        nodes: List[BaseNode],
        similarity_top_k: int,
        corpus_tokenizer: Callable[[str], List[str]],         # 문서는 리스트 반환
        query_tokenizer: Callable[[str], Dict[str, float]]    # 🚨 질문은 딕셔너리 반환!
    ) -> None:
        self._nodes = nodes
        self._similarity_top_k = similarity_top_k
        self._corpus_tokenizer = corpus_tokenizer
        self._query_tokenizer = query_tokenizer

        print("🚀 [Index] 문서 인덱싱 시작 (Kiwi)...")
        # 문서는 기존대로 토큰 리스트로 변환하여 인덱싱
        corpus_tokens = [self._corpus_tokenizer(node.text) for node in nodes]

        self._bm25 = bm25s.BM25()
        self._bm25.index(corpus_tokens)
        print("✅ [Index] 인덱싱 완료!")

        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        query_str = query_bundle.query_str

        # 1. 쿼리 토크나이징 (이제 딕셔너리를 받습니다!)
        # 예: {'시행착오': 1.0, '시오법': 0.7}
        weighted_query = self._query_tokenizer(query_str)

        # 2. [핵심 로직 이식] 가중치 기반 검색 (WeightedBM25S_Final 로직)
        doc_scores = defaultdict(float)

        # 단어 하나씩 검색해서 가중치 곱해서 더하기
        for token, weight in weighted_query.items():
            try:
                # bm25s는 입력을 이중 리스트로 받음 [[token]]
                results = self._bm25.retrieve([[token]], k=len(self._nodes))
            except Exception:
                continue

            if results.documents.size == 0:
                continue

            indices = results.documents[0]
            scores = results.scores[0]

            # (BM25점수 * 우리가 정한 가중치) 누적
            for idx, score in zip(indices, scores):
                doc_scores[idx] += (score * weight)

        # 3. 점수순 정렬
        sorted_docs = sorted(doc_scores.items(), key=lambda x: x[1], reverse=True)

        # 4. 상위 k개만 잘라서 LlamaIndex 포맷(NodeWithScore)으로 변환
        top_k_docs = sorted_docs[:self._similarity_top_k]

        nodes_with_scores = []
        for idx, score in top_k_docs:
            nodes_with_scores.append(
                NodeWithScore(node=self._nodes[idx], score=float(score))
            )

        return nodes_with_scores

    # persist 함수는 기존과 동일하게 유지 가능
    def persist(self, path: str):
        self._bm25.save(path)
        print(f"💾 인덱스 저장 완료: {path}")

In [ ]:
from functools import partial

# 1. 쿼리 토크나이저 다시 조립 (Dict 반환하도록 수정한 버전 적용)
query_tokenizer_rich = partial(
    tokenize_query_rich, # 🚨 1단계에서 수정한(Dict반환) 함수여야 함!
    tokenizer_obj=tokenizer,
    l2m_map_obj=l2m_map,
    graph_retriever_obj=graph_retriever
)

kiwi_instance = Kiwi()
# 2. 문서 토크나이저 (기존 Kiwi - List 반환)
corpus_tokenizer = partial(
    tokenize_kiwi,
    kiwi=kiwi_instance,
    tag_include=kiwi_tags,
    text_type="corpus",
    top_n=2,
    score_threshold=1.2,
)

# 3. [최종] 통합 리트리버 생성
final_retriever = KiwiWeightedBM25Retriever(
    nodes=nodes,
    similarity_top_k=30,
    corpus_tokenizer=corpus_tokenizer,  # 문서는 kiwi
    query_tokenizer=query_tokenizer_rich # 질문은 kiwi+ner+유의어
)

print("🎉 완벽한 하이브리드 가중치 검색기 완성!")

In [ ]:
# 1. 타겟 설정 (아까 찾은 대박 사례)
target_idx = 306
target_doc_id = 52478

# 2. 데이터셋에서 정보 가져오기
item = dataset['train'][target_idx]
question = item['question']
# 정답은 보통 리스트 형태({'text': ['정답'], 'answer_start': [...]})로 들어있습니다.
gt_answer = item['answers']['text'][0]

print(f" 질문 (Q{target_idx}): {question}")
print(f" 정답 (Ground Truth): **{gt_answer}**")
print("=" * 60)

# 3. 문서 내용 가져오기 (완전체)
full_text = ""
for node in nodes:
    did = node.metadata.get('docid') or node.metadata.get('document_id')
    if did and int(did) == target_doc_id:
        full_text += node.get_content() + "\n" # 청크 이어붙이기

# 4. 검증 결과 출력
if not full_text:
    print(" 문서를 못 찾았습니다.")
else:
    # 정답이 문서에 포함되어 있는지 확인
    if gt_answer in full_text:
        print(f" [검증 성공] 찾은 문서에 정답이 포함되어 있습니다! ")

        # 시각적으로 보여주기 (정답 부분 하이라이트)
        # 정답 주변 50자만 잘라서 보여줌
        idx = full_text.find(gt_answer)
        start = max(0, idx - 50)
        end = min(len(full_text), idx + len(gt_answer) + 50)

        snippet = full_text[start:end].replace('\n', ' ')

        print(f"📝 문서 내용 발췌:")
        print(f"   ...{snippet.replace(gt_answer, f'[{gt_answer}]')}...")

        print("-" * 60)
        print(" 결론:")
        print("   Basic 검색기는 '본래'만 찾다가 이 문서를 놓쳤지만,")
        print("   Rich 검색기는 '원래'를 유의어로 확장해서 이 문서를 찾았고,")
        print("   결국 **정답(연기배우...)을 맞출 수 있는 근거**를 확보했습니다!")

    else:
        print(f" [검증 실패] 문서는 찾았는데 텍스트가 정확히 일치하진 않습니다.")
        print(f"   (사람 눈으로 확인 필요. 의미는 같지만 표현이 다를 수 있음)")
        print(f"   문서 앞부분: {full_text[:100]}...")

In [ ]:
# 분석할 타겟 문서 ID (Rich 1등, Basic 5등이었던 그 녀석)
target_doc_id = 51638
target_query = dataset['train'][1]['question']

print(f"🎯 타겟 분석: 문서 {target_doc_id} vs 질문 '{target_query}'")
print("=" * 60)

# 1. 문서 내용 가져오기 (Rich 결과에서 찾기)
# (이미 검색된 결과 res_rich 리스트에서 찾습니다)
target_node = None
for node_item in res_rich:
    did = node_item.node.metadata.get('docid') or node_item.node.metadata.get('document_id')
    if int(did) == target_doc_id:
        target_node = node_item.node
        break

if not target_node:
    print("😱 Rich 검색 결과에서 해당 문서를 찾을 수 없습니다!")
else:
    doc_content = target_node.get_content().replace('\n', ' ')
    print(f"📝 문서 내용 (일부): {doc_content[:100]}...")
    print("-" * 60)

    # 2. 쿼리 분석 (Rich 토크나이저 사용)
    # 토큰과 점수를 가져옵니다.
    token_scores = retriever_rich._query_tokenizer(target_query)

    # 원본단어(Score >= 1.0) vs 유의어(Score < 1.0) 분리
    original_tokens = {t for t, s in token_scores.items() if s >= 1.0}
    synonym_tokens = {t for t, s in token_scores.items() if s < 1.0}

    # 3. 매칭 검사 (Hit Check)
    hit_originals = [t for t in original_tokens if t in doc_content]
    hit_synonyms = [t for t in synonym_tokens if t in doc_content]

    # 4. 시각화 출력
    print(f"📊 [Basic 관점] 원본 단어 매칭: {len(hit_originals)}개")
    print(f"   👉 목록: {hit_originals}")
    print(f"   (Basic에서는 이 단어들만 보여서 점수가 낮았음)")

    print("-" * 30)

    print(f"🚀 [Rich 관점] 유의어 추가 매칭: {len(hit_synonyms)}개")
    print(f"   👉 목록: {hit_synonyms}")
    print(f"   (이 단어들이 추가 점수를 줘서 1등으로 밀어올림!)")

    print("-" * 30)

    # 5. 문맥 하이라이트 (유의어가 쓰인 문장 찾기)
    if hit_synonyms:
        print("👀 [결정적 문맥] 유의어가 등장한 부분:")
        for syn in hit_synonyms:
            idx = doc_content.find(syn)
            start = max(0, idx - 20)
            end = min(len(doc_content), idx + 20)
            print(f"   🔥 '{syn}': ...{doc_content[start:end]}...")
    else:
        print("   (유의어가 본문에 직접 등장하진 않았습니다. 복합명사 등 다른 요인일 수 있음)")

print("=" * 60)

In [ ]:
from google.colab import drive
drive.mount('/content/drive') # 이미 마운트되어 있으면 패스

# 구글 드라이브 내의 원하는 경로 지정
save_path = "/content/drive/MyDrive/vllm_baseline/my_search_index"

# 저장 실행!
final_retriever.persist(save_path)

print(f"✅ 구글 드라이브에 안전하게 저장했습니다: {save_path}")

In [ ]:
from llama_index.core import Settings

# 🛑 "나 OpenAI 키 없어! 기본 LLM으로 OpenAI 찾지 마!" 라고 선언
Settings.llm = None
# (만약 로드해둔 Gemma 모델 변수 'llm'이 있다면 Settings.llm = llm 이라고 넣으셔도 됩니다)

print("✅ 기본 LLM 설정을 'None'으로 변경했습니다. (OpenAI 에러 해결)")

In [ ]:
from llama_index.core.retrievers import QueryFusionRetriever

# 1. Dense Retriever (벡터 검색 - 의미 기반)
# (Top-k를 넉넉하게 50~100개 가져오는 게 좋습니다. 섞을 때 후보가 많아야 좋거든요.)
dense_retriever = vector_index.as_retriever(similarity_top_k=50)

# 2. Sparse Retriever (우리가 만든 하이브리드 - 키워드 기반)
# (이 변수 이름이 아까 만든 final_retriever 인지 꼭 확인하세요!)
sparse_retriever = final_retriever

print("🔗 Fusion Retriever (RRF 방식) 연결 중...")

fusion_retriever = QueryFusionRetriever(
    retrievers=[dense_retriever, sparse_retriever], # 두 개를 리스트로 전달
    similarity_top_k=30,          # 최종적으로 사용자에게 보여줄 개수
    num_queries=1,                # 쿼리 생성 안 함 (원본 질문 그대로 사용)
    use_async=False,
    mode="reciprocal_rerank"      # 🌟 핵심: 등수 기반으로 공평하게 섞기
)

print("✅ 하이브리드 검색 준비 완료!")

In [ ]:
# 1. 테스트할 질문 가져오기
query = dataset['train'][123]['question']
print(f"❓ 질문: {query}")
print("-" * 60)

# ---------------------------------------------------------
# 2. [추가됨] 쿼리 토크나이징 결과 미리보기 👀
# ---------------------------------------------------------
target_retriever = final_retriever

print(f"🧩 [{target_retriever.__class__.__name__}] 쿼리 분석 결과")

try:
    # 리트리버 안에 숨겨진 토크나이저 함수를 꺼냅니다.
    tokenizer_func = target_retriever._query_tokenizer

    # 질문을 넣어서 결과를 봅니다.
    tokenized_result = tokenizer_func(query)

    # 결과가 딕셔너리(가중치 포함)인지 리스트인지에 따라 예쁘게 출력
    if isinstance(tokenized_result, dict):
        print(f"   👉 타입: 가중치 딕셔너리 (Token: Weight)")
        print(f"   👉 키워드 개수: {len(tokenized_result)}개")
        print(f"   👉 내용: {tokenized_result}")
    else:
        print(f"   👉 타입: 토큰 리스트")
        print(f"   👉 내용: {tokenized_result}")

except Exception as e:
    print(f"   ⚠️ 토크나이저 확인 중 에러 발생: {e}")
    print("   (혹시 FusionRetriever를 target으로 하셨나요? 그렇다면 내부의 sparse_retriever를 지정해야 합니다.)")

print("-" * 60)

# ---------------------------------------------------------
# 3. 검색 실행
# ---------------------------------------------------------
print(f"🚀 검색기 실행 중...")
results = target_retriever.retrieve(query)

# ---------------------------------------------------------
# 4. 결과 출력
# ---------------------------------------------------------
print(f"\n🔍 검색 결과: 총 {len(results)}개 발견\n")

for i, node_with_score in enumerate(results[:5]): # 상위 5개만 확인
    node = node_with_score.node
    score = node_with_score.score

    # 줄바꿈 제거
    clean_content = node.get_content().replace('\n', ' ')

    # 문서 ID 안전하게 가져오기
    doc_id = node.metadata.get('docid') or node.metadata.get('document_id') or node.node_id

    print(f"🏅 [Rank {i+1}] Score: {score:.4f}")
    print(f"📄 문서 ID: {doc_id}")
    print(f"📝 내용: {clean_content[:100]}...")
    print("-" * 60)

In [ ]:
# 1. 테스트할 질문 가져오기
query = dataset['train'][307]['question']
print(f"❓ 질문: {query}")
print("-" * 60)

# ---------------------------------------------------------
# 2. 검색 실행 (Sparse / Fusion 선택)
# ---------------------------------------------------------
# final_retriever: 우리가 만든 Kiwi+GLiNER+Graph 검색기 (Sparse)
# fusion_retriever: Dense + Sparse 합친 것 (Fusion)
target_retriever = final_retriever

print(f"🚀 검색기 실행 중... ({target_retriever.__class__.__name__})")
results = target_retriever.retrieve(query)

# ---------------------------------------------------------
# 3. 결과 깔끔하게 출력하기 (요청하신 포맷)
# ---------------------------------------------------------
print(f"\n🔍 검색 결과: 총 {len(results)}개 발견\n")

for i, node_with_score in enumerate(results[:5]): # 상위 5개만 확인
    node = node_with_score.node
    score = node_with_score.score

    # 내용에서 줄바꿈(\n)을 공백으로 변경하여 한 줄로 보기 좋게 만듦
    clean_content = node.get_content().replace('\n', ' ')

    print(f"🏅 [Rank {i+1}] Score: {score:.4f}")

    # 문서 ID (metadata에 없으면 node_id 사용)
    doc_id = node.metadata.get('docid') or node.metadata.get('document_id') or node.node_id
    print(f"📄 문서 ID: {doc_id}")

    # 내용은 100자까지만 출력
    print(f"📝 내용: {clean_content[:100]}...")
    print("-" * 60)

In [ ]:
# 1. 테스트할 질문 가져오기
query = dataset['train'][307]['question']
print(f"❓ 질문: {query}")
print("-" * 60)

# ---------------------------------------------------------
# 2. 검색 실행 (Sparse / Fusion 선택)
# ---------------------------------------------------------
# final_retriever: 우리가 만든 Kiwi+GLiNER+Graph 검색기 (Sparse)
# fusion_retriever: Dense + Sparse 합친 것 (Fusion)
target_retriever = fusion_retriever

print(f"🚀 검색기 실행 중... ({target_retriever.__class__.__name__})")
results = target_retriever.retrieve(query)

# ---------------------------------------------------------
# 3. 결과 깔끔하게 출력하기 (요청하신 포맷)
# ---------------------------------------------------------
print(f"\n🔍 검색 결과: 총 {len(results)}개 발견\n")

for i, node_with_score in enumerate(results[:5]): # 상위 5개만 확인
    node = node_with_score.node
    score = node_with_score.score

    # 내용에서 줄바꿈(\n)을 공백으로 변경하여 한 줄로 보기 좋게 만듦
    clean_content = node.get_content().replace('\n', ' ')

    print(f"🏅 [Rank {i+1}] Score: {score:.4f}")

    # 문서 ID (metadata에 없으면 node_id 사용)
    doc_id = node.metadata.get('docid') or node.metadata.get('document_id') or node.node_id
    print(f"📄 문서 ID: {doc_id}")

    # 내용은 100자까지만 출력
    print(f"📝 내용: {clean_content[:100]}...")
    print("-" * 60)

In [ ]:
query = dataset['train'][86]['question']
retrieved_nodes = fusion_retriever.retrieve(query)

docs_for_rerank = [n.node.text for n in retrieved_nodes]
ids_for_rerank = [n.node.metadata['document_id'] for n in retrieved_nodes]

reranked_results = reranker.rerank(query, docs_for_rerank, ids_for_rerank, top_k=5)

final_context = "\n\n---\n\n".join([f"[{i+1}] {d[0]}" for i, d in enumerate(reranked_results[0])])
print(f'Q. {query}')

print(reranked_results[1])


In [ ]:
query = dataset['train'][86]['question']
retrieved_nodes = dense_retriever.retrieve(query)

docs_for_rerank = [n.node.text for n in retrieved_nodes]
ids_for_rerank = [n.node.metadata['document_id'] for n in retrieved_nodes]

reranked_results = reranker.rerank(query, docs_for_rerank, ids_for_rerank, top_k=5)

final_context = "\n\n---\n\n".join([f"[{i+1}] {d[0]}" for i, d in enumerate(reranked_results[0])])
print(f'Q. {query}')

print(reranked_results[1])


In [ ]:
# 1. 테스트할 질문 가져오기
query = dataset['train'][307]['question']
print(f"❓ 질문: {query}")
print("-" * 60)

# 2. GLiNer 단독 검색 실행
# (내부에서 print(f"🔍 Query Tokens: ...") 로그가 찍힐 겁니다. 이걸 꼭 확인하세요!)
gliner_results = dense_retriever.retrieve(query)

# 3. 결과 뜯어보기
print(f"\n🔍 [Dense Only] 검색 결과: {len(gliner_results)}개 발견\n")

for i, node_with_score in enumerate(gliner_results[:5]): # 상위 5개만
    node = node_with_score.node
    score = node_with_score.score

    print(f"🏅 [Rank {i+1}] BM25 Score: {score:.4f}")
    print(f"📄 문서 ID: {node.metadata.get('document_id', 'N/A')}")
    # 가독성을 위해 줄바꿈 제거하고 100자만 출력
    print(f"📝 내용: {node.text[:100].replace('\n', ' ')}...")
    print("-" * 60)

In [ ]:
valid_set_dir = "/data/train_dataset"
valid_dataset = load_from_disk(valid_set_dir)

In [ ]:
# 1. 여기를 바꿔주세요! (notebook 전용 tqdm 사용)
from tqdm.notebook import tqdm
import json
import numpy as np

result_for_test = []

print("🚀 Validation 데이터 평가 시작...")

# 2. tqdm(...) 으로 감싸면 됩니다. (tqdm.tqdm 아님)
for i in tqdm(range(len(valid_dataset['validation']))):

    # 질문과 id
    test_q_query = valid_dataset['validation'][i]['question']
    test_q_id = valid_dataset['validation'][i]['id']

    # 골든리트리버 귀엽다 (Fusion 검색)
    retrieved_nodes_test = fusion_retriever.retrieve(test_q_query)

    # data for reranker
    docs_for_rerank_test = [n.node.text for n in retrieved_nodes_test]
    # metadata 안전하게 가져오기
    ids_for_rerank_test = [n.node.metadata.get('document_id', -1) for n in retrieved_nodes_test]

    # rerank result
    reranked_results_test = reranker.rerank(test_q_query, docs_for_rerank_test, ids_for_rerank_test, top_k=5)

    # ID 추출 및 int 변환
    final_doc_ids = [int(did) for did in reranked_results_test[1]]

    result_for_test.append([test_q_id, final_doc_ids])

# ---------------------------------------------------------
# JSON 변환 및 저장
# ---------------------------------------------------------
def convert_to_json(data):
    question_ids = []
    document_lists = []

    for q_id, doc_list in data:
        question_ids.append(q_id)
        document_lists.append(doc_list)
    result_dict = {
        "question_id": question_ids,
        "document_id": document_lists
    }
    return result_dict

json_test = convert_to_json(result_for_test)

file_path = '/kiwi+ner+유의어 결과/validation_ner_hybrid.json'

import os
os.makedirs(os.path.dirname(file_path), exist_ok=True)

with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(json_test, f, ensure_ascii=False, indent=4)

print(f"💾 저장 완료! 경로: {file_path}")

In [ ]:
test_set_dir = "/data/test_dataset"
test_dataset = load_from_disk(test_set_dir)

In [ ]:
print(test_dataset['validation'][0])

In [ ]:
from tqdm.notebook import tqdm
import json
import numpy as np
import os

# 결과를 담을 리스트 (이름 변경)
test_results_final = []

print("🚀 Test 데이터 평가 시작...")

# 1. 데이터셋 루프 (valid_dataset -> test_dataset['test'] 로 변경)
for i in tqdm(range(len(test_dataset['validation']))):

    # 질문과 id 가져오기
    target_query = test_dataset['validation'][i]['question']
    target_id = test_dataset['validation'][i]['id']

    # 2. 골든리트리버 (Fusion 검색)
    retrieved_nodes = fusion_retriever.retrieve(target_query)

    # 3. Reranker용 데이터 준비
    docs_for_rerank = [n.node.text for n in retrieved_nodes]
    # metadata 안전하게 가져오기
    ids_for_rerank = [n.node.metadata.get('document_id', -1) for n in retrieved_nodes]

    # 4. Rerank 실행
    reranked_results = reranker.rerank(target_query, docs_for_rerank, ids_for_rerank, top_k=5)

    # 5. ID 추출 및 int 변환
    # reranked_results[1]에 이미 중복 제거된 ID 리스트가 들어있음
    final_doc_ids = [int(did) for did in reranked_results[1]]

    test_results_final.append([target_id, final_doc_ids])

# ---------------------------------------------------------
# JSON 변환 및 저장
# ---------------------------------------------------------
def convert_to_json(data):
    question_ids = []
    document_lists = []

    for q_id, doc_list in data:
        question_ids.append(q_id)
        document_lists.append(doc_list)

    result_dict = {
        "question_id": question_ids,
        "document_id": document_lists
    }
    return result_dict

# 변환
json_output = convert_to_json(test_results_final)

# 💾 파일명 변경: validation -> test
save_path = '/kiwi+ner+유의어 결과/test_ner_synonym_hybrid.json'

# 폴더 생성 (안전장치)
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# 저장
with open(save_path, 'w', encoding='utf-8') as f:
    json.dump(json_output, f, ensure_ascii=False, indent=4)

print(f"💾 Test 결과 저장 완료! 경로: {save_path}")

In [ ]:
# 1. 여기를 바꿔주세요! (notebook 전용 tqdm 사용)
# 일어나서 다시 돌리기
from tqdm.notebook import tqdm
import json
import numpy as np

# 🚨 저장할 리스트 이름 확인
result_for_test_only = []

print("🚀 Validation 데이터 평가 시작 (Sparse Only)...")

# 2. tqdm(...) 으로 감싸면 됩니다. (tqdm.tqdm 아님)
for i in tqdm(range(len(valid_dataset['validation']))):

    # 질문과 id
    test_q_query = valid_dataset['validation'][i]['question']
    test_q_id = valid_dataset['validation'][i]['id']

    # 🚨 final_retriever (Sparse Only) 사용 확인
    retrieved_nodes_test = final_retriever.retrieve(test_q_query)

    # data for reranker
    docs_for_rerank_test = [n.node.text for n in retrieved_nodes_test]
    # metadata 안전하게 가져오기
    ids_for_rerank_test = [n.node.metadata.get('document_id', -1) for n in retrieved_nodes_test]

    # rerank result
    reranked_results_test = reranker.rerank(test_q_query, docs_for_rerank_test, ids_for_rerank_test, top_k=5)

    # ID 추출 및 int 변환
    final_doc_ids = [int(did) for did in reranked_results_test[1]]

    # 리스트에 추가
    result_for_test_only.append([test_q_id, final_doc_ids])

# ---------------------------------------------------------
# JSON 변환 및 저장
# ---------------------------------------------------------
def convert_to_json(data):
    question_ids = []
    document_lists = []

    for q_id, doc_list in data:
        question_ids.append(q_id)
        document_lists.append(doc_list)
    result_dict = {
        "question_id": question_ids,
        "document_id": document_lists
    }
    return result_dict

# 🚨 [수정 완료] 위에서 만든 리스트(result_for_test_only)를 넣어야 합니다!
json_test = convert_to_json(result_for_test_only)

file_path = '/kiwi+ner+유의어 결과/validation_ner_synonym_only.json'

import os
os.makedirs(os.path.dirname(file_path), exist_ok=True)

with open(file_path, 'w', encoding='utf-8') as f:
    json.dump(json_test, f, ensure_ascii=False, indent=4)

print(f"💾 저장 완료! 경로: {file_path}")

In [ ]:
from tqdm.notebook import tqdm
import json
import numpy as np
import os

# 🚨 변수명 변경: _only를 붙여서 이전 변수와 겹치지 않게 함
test_results_final_only = []

print("🚀 Test 데이터 평가 시작 (Sparse Only)...")

# 1. 데이터셋 루프
for i in tqdm(range(len(test_dataset['validation']))):

    # 질문과 id 가져오기
    target_query = test_dataset['validation'][i]['question']
    target_id = test_dataset['validation'][i]['id']

    # 2. 골든리트리버 (final_retriever = Sparse Only)
    # 🚨 여기가 fusion_retriever가 아닌지 꼭 확인! (현재 코드는 final_retriever라 맞음)
    retrieved_nodes = final_retriever.retrieve(target_query)

    # 3. Reranker용 데이터 준비
    docs_for_rerank = [n.node.text for n in retrieved_nodes]
    # metadata 안전하게 가져오기
    ids_for_rerank = [n.node.metadata.get('document_id', -1) for n in retrieved_nodes]

    # 4. Rerank 실행
    reranked_results = reranker.rerank(target_query, docs_for_rerank, ids_for_rerank, top_k=5)

    # 5. ID 추출 및 int 변환
    # reranked_results[1]에 이미 중복 제거된 ID 리스트가 들어있음
    final_doc_ids = [int(did) for did in reranked_results[1]]

    # 🚨 변경된 리스트에 추가
    test_results_final_only.append([target_id, final_doc_ids])

# ---------------------------------------------------------
# JSON 변환 및 저장
# ---------------------------------------------------------
def convert_to_json(data):
    question_ids = []
    document_lists = []

    for q_id, doc_list in data:
        question_ids.append(q_id)
        document_lists.append(doc_list)

    result_dict = {
        "question_id": question_ids,
        "document_id": document_lists
    }
    return result_dict

# 🚨 변경된 리스트를 변환
json_output = convert_to_json(test_results_final_only)

# 💾 파일명 설정
save_path = '/kiwi+ner+유의어 결과/test_Ner_synonym_only.json'

# 폴더 생성 (안전장치)
os.makedirs(os.path.dirname(save_path), exist_ok=True)

# 저장
with open(save_path, 'w', encoding='utf-8') as f:
    json.dump(json_output, f, ensure_ascii=False, indent=4)

print(f"💾 Test 결과(Only) 저장 완료! 경로: {save_path}")

아래는 백업 후 자료 확보를 위한 코드입니다.

In [ ]:
import pickle
import os
import bm25s
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Document
from functools import partial
from kiwipiepy import Kiwi

# =========================================================
# 1. Nodes(본문) 다시 만들기 (CPU 작업 - 약 2~3분 소요)
# =========================================================
# 2. 도구(Tokenizer) 다시 조립
# =========================================================
# (위에서 이미 tokenizer, l2m_map, graph_retriever 객체는 생성되어 있어야 함)
print("🚀 2. 토크나이저 조립 중...")

query_tokenizer_rich = partial(
    tokenize_query_rich,
    tokenizer_obj=tokenizer,
    l2m_map_obj=l2m_map,
    graph_retriever_obj=graph_retriever
)

kiwi_instance = Kiwi()
corpus_tokenizer = partial(
    tokenize_kiwi,
    kiwi=kiwi_instance,
    tag_include=kiwi_tags,
    text_type="corpus",
    top_n=2,
    score_threshold=1.2,
)
# =========================================================
# 3. [핵심] 저장된 인덱스 불러와서 합체 (Indexing 스킵!)
# =========================================================
print("🚀 3. 저장된 BM25 인덱스 로드 중...")

# 저장된 인덱스 파일 로드
loaded_bm25 = bm25s.BM25.load(save_path, load_corpus=False)

# 껍데기만 생성 (__new__ 사용) -> __init__ 실행 안 함
final_retriever = KiwiWeightedBM25Retriever.__new__(KiwiWeightedBM25Retriever)

# 부품 수동 조립
final_retriever._nodes = nodes            # 방금 만든 Nodes
final_retriever._bm25 = loaded_bm25       # 저장해둔 인덱스
final_retriever._query_tokenizer = query_tokenizer_rich # 질문 처리기
final_retriever._similarity_top_k = 30
final_retriever._corpus_tokenizer = None  # 검색땐 필요 없으므로 None 처리

print("🎉 final_retriever 부활 완료! (30분 기다릴 필요 없음)")

In [ ]:
# 1) 그래프 검색기: 문지기를 1.0으로 높여서 유의어 차단
graph_retriever_basic = GraphRetriever(synonym_graph, min_weight=1.0)

# 2) 토크나이저: 기본 그래프 검색기 장착
tokenizer_basic = partial(
    tokenize_query_rich,
    tokenizer_obj=tokenizer,
    l2m_map_obj=l2m_map,
    graph_retriever_obj=graph_retriever_basic, # 🚨 1.0 짜리
)

# 3) 리트리버 조립
retriever_basic = KiwiWeightedBM25Retriever.__new__(KiwiWeightedBM25Retriever)
retriever_basic._nodes = nodes
retriever_basic._bm25 = loaded_bm25
retriever_basic._query_tokenizer = tokenizer_basic
retriever_basic._similarity_top_k = 30
retriever_basic._corpus_tokenizer = None

# =========================================================
# 🚀 2. [Rich] 확장 검색기 생성 (유의어 O)
# =========================================================
print("🐇 2. [Rich] 확장 검색기 조립 중 (min_weight=0.7)...")

# 1) 그래프 검색기: 문지기를 0.7로 낮춰서 유의어 통과
graph_retriever_rich = GraphRetriever(synonym_graph, min_weight=0.7)

# 2) 토크나이저: 확장 그래프 검색기 장착
tokenizer_rich = partial(
    tokenize_query_rich,
    tokenizer_obj=tokenizer,
    l2m_map_obj=l2m_map,
    graph_retriever_obj=graph_retriever_rich, # 🚨 0.7 짜리
)

# 3) 리트리버 조립
retriever_rich = KiwiWeightedBM25Retriever.__new__(KiwiWeightedBM25Retriever)
retriever_rich._nodes = nodes
retriever_rich._bm25 = loaded_bm25
retriever_rich._query_tokenizer = tokenizer_rich
retriever_rich._similarity_top_k = 30
retriever_rich._corpus_tokenizer = None



In [ ]:
import pandas as pd
from tqdm.notebook import tqdm

# -------------------------------------------------------
# 1. 수색 범위 설정
# -------------------------------------------------------
start_idx = 306
end_idx = 307  # 200개만 먼저 봅시다
improvement_cases = []

print(f"🕵️‍♂️ 심층 수색 시작 (토크나이징 비교 포함): {start_idx} ~ {end_idx}")

# -------------------------------------------------------
# 2. 수색 루프
# -------------------------------------------------------
for i in tqdm(range(start_idx, end_idx)):
    if i >= len(dataset['train']): break

    item = dataset['train'][i]
    query = item['question']
    gt_doc_id = int(item['document_id'])

    # ---------------------------------------------------
    # 🔍 [NEW] 토크나이징 결과 미리 캡처!
    # ---------------------------------------------------
    # A. Basic 토큰 (threshold=1.0 이라 원본만 나옴)
    tokens_basic_raw = retriever_basic._query_tokenizer(query)
    tokens_basic_list = list(tokens_basic_raw.keys()) # 키만 가져옴

    # B. Rich 토큰 (threshold=0.0 이라 유의어 포함)
    tokens_rich_raw = retriever_rich._query_tokenizer(query)
    # 가독성을 위해 유의어는 '단어(0.7)' 형태로 포맷팅
    tokens_rich_fmt = []
    for t, s in tokens_rich_raw.items():
        if s < 1.0:
            tokens_rich_fmt.append(f"{t}({s:.1f})") # 유의어 표시
        else:
            tokens_rich_fmt.append(t) # 원본

    # ---------------------------------------------------
    # 3. 검색 실행
    # ---------------------------------------------------
    res_basic = retriever_basic.retrieve(query)
    ids_basic = [int(n.node.metadata.get('docid') or n.node.metadata.get('document_id')) for n in res_basic]

    res_rich = retriever_rich.retrieve(query)
    ids_rich = [int(n.node.metadata.get('docid') or n.node.metadata.get('document_id')) for n in res_rich]

    # 순위 계산
    basic_rank = ids_basic.index(gt_doc_id) if gt_doc_id in ids_basic else 100
    rich_rank = ids_rich.index(gt_doc_id) if gt_doc_id in ids_rich else 100

    # ---------------------------------------------------
    # 4. 조건 검사: Rich가 더 잘했는가?
    # ---------------------------------------------------
    if rich_rank < basic_rank:

        # 유의어가 실제로 문서에 있는지 확인
        synonyms = {t for t, s in tokens_rich_raw.items() if s < 1.0}

        target_node = next((n.node for n in res_rich if int(n.node.metadata.get('docid') or n.node.metadata.get('document_id')) == gt_doc_id), None)

        if target_node:
            doc_text = target_node.get_content()
            hit_synonyms = [syn for syn in synonyms if syn in doc_text]

            # 유의어가 하나라도 적중했거나, 혹은 순위가 드라마틱하게 올랐다면 저장
            if hit_synonyms or (basic_rank > 10 and rich_rank <= 5):
                improvement_cases.append({
                    'index': i,
                    'query': query,
                    'doc_id': gt_doc_id,
                    'basic_rank': basic_rank + 1,
                    'rich_rank': rich_rank + 1,
                    'diff': basic_rank - rich_rank,
                    'hit_synonyms': hit_synonyms,
                    'snippet': doc_text[:80].replace('\n', ' '),
                    # 📝 토큰 정보 저장
                    'tokens_basic': tokens_basic_list,
                    'tokens_rich': tokens_rich_fmt
                })

# -------------------------------------------------------
# 5. 결과 발표
# -------------------------------------------------------
improvement_cases.sort(key=lambda x: x['diff'], reverse=True)

print(f"\n 총 {len(improvement_cases)}개의 사례 발견!\n")

for case in improvement_cases[:5]: # 상위 5개
    print(f" [Q{case['index']}] {case['query']}")
    print(f"   📄 정답 ID: {case['doc_id']}")

    b_rank = "순위권 밖" if case['basic_rank'] > 30 else f"{case['basic_rank']}위"
    print(f"    순위 변화: Basic({b_rank}) -> Rich({case['rich_rank']}위) ( {case['diff']} 상승)")

    print(f"    [Basic 토큰]: {case['tokens_basic']}")
    print(f"    [Rich 토큰] : {case['tokens_rich']}")

    if case['hit_synonyms']:
        print(f"    [적중 유의어]: {case['hit_synonyms']}")
    else:
        print(f"    (유의어 직접 매칭은 없지만, 다른 텀 가중치 변경으로 상승 추정)")

    print(f"   📝 문서: {case['snippet']}...")
    print("=" * 60)

In [ ]:
import pandas as pd
from tqdm.notebook import tqdm

# -------------------------------------------------------
# 1. 수색 범위 설정
# -------------------------------------------------------
start_idx = 306
end_idx = 307
print(f"[System] Search started: Index {start_idx} ~ {end_idx}")

# -------------------------------------------------------
# 2. 수색 루프
# -------------------------------------------------------
for i in tqdm(range(start_idx, end_idx)):
    if i >= len(dataset['train']): break

    item = dataset['train'][i]
    query = item['question']
    gt_doc_id = int(item['document_id'])
    gt_answer = item['answers']['text'][0] # 정답 텍스트

    # ---------------------------------------------------
    # A. 토크나이징 결과 캡처
    # ---------------------------------------------------
    # Basic (유의어 X)
    tokens_basic_raw = retriever_basic._query_tokenizer(query)
    tokens_basic_list = list(tokens_basic_raw.keys())

    # Rich (유의어 O)
    tokens_rich_raw = retriever_rich._query_tokenizer(query)
    tokens_rich_fmt = []
    rich_synonyms_only = set()

    for t, s in tokens_rich_raw.items():
        if s < 1.0:
            tokens_rich_fmt.append(f"{t}({s:.1f})")
            rich_synonyms_only.add(t)
        else:
            tokens_rich_fmt.append(t)

    # ---------------------------------------------------
    # B. 검색 실행 및 순위 계산
    # ---------------------------------------------------
    res_basic = retriever_basic.retrieve(query)
    ids_basic = [int(n.node.metadata.get('docid') or n.node.metadata.get('document_id')) for n in res_basic]

    res_rich = retriever_rich.retrieve(query)
    ids_rich = [int(n.node.metadata.get('docid') or n.node.metadata.get('document_id')) for n in res_rich]

    # 순위 (없으면 100위 처리)
    basic_rank = ids_basic.index(gt_doc_id) if gt_doc_id in ids_basic else 100
    rich_rank = ids_rich.index(gt_doc_id) if gt_doc_id in ids_rich else 100

    # ---------------------------------------------------
    # C. 조건 검사: Rich 순위가 Basic보다 높은가?
    # ---------------------------------------------------
    if rich_rank < basic_rank:

        # ---------------------------------------------------
        # D. 문서 전체 내용(Full Text) 가져오기
        # ---------------------------------------------------
        # 검색된 청크만 보는게 아니라, 해당 ID의 원본 전체를 가져옵니다.
        full_doc_text = ""
        found_chunks = 0

        for node in nodes:
            did = node.metadata.get('docid') or node.metadata.get('document_id')
            if did and int(did) == gt_doc_id:
                full_doc_text += node.get_content() + "\n"
                found_chunks += 1

        if not full_doc_text:
            full_doc_text = "(Error: Document not found in nodes list)"

        # ---------------------------------------------------
        # E. 상세 분석 (유의어 & 정답 매칭)
        # ---------------------------------------------------

        # 1) 유의어 적중 확인
        hit_synonyms = [syn for syn in rich_synonyms_only if syn in full_doc_text]

        # 2) 정답 포함 여부 확인
        has_answer = gt_answer in full_doc_text

        # 3) 출력용 스니펫 생성 (정답 주변 or 유의어 주변)
        snippet = ""
        if has_answer:
            idx = full_doc_text.find(gt_answer)
            start = max(0, idx - 40)
            end = min(len(full_doc_text), idx + len(gt_answer) + 40)
            snippet = full_doc_text[start:end].replace('\n', ' ')
            # 정답 강조 표시
            snippet = snippet.replace(gt_answer, f" **[{gt_answer}]** ")
        elif hit_synonyms:
            # 정답은 못 찾았지만 유의어가 있는 경우 유의어 주변 표시
            idx = full_doc_text.find(hit_synonyms[0])
            start = max(0, idx - 40)
            end = min(len(full_doc_text), idx + 20)
            snippet = full_doc_text[start:end].replace('\n', ' ')
        else:
            snippet = full_doc_text[:100].replace('\n', ' ')

        # ---------------------------------------------------
        # F. 결과 출력 (이모티콘 제외)
        # ---------------------------------------------------
        print("=" * 80)
        print(f"[Case Analysis] Q{i} : {query}")
        print(f" - Doc ID: {gt_doc_id}")
        print(f" - Ground Truth: {gt_answer}")
        print("-" * 80)

        print(f"[Rank Change]")
        b_rank_str = "Out of Rank (>30)" if basic_rank > 30 else f"{basic_rank}"
        print(f" - Basic: {b_rank_str} -> Rich: {rich_rank} (Diff: +{basic_rank - rich_rank})")

        print(f"[Token Analysis]")
        print(f" - Basic Tokens: {tokens_basic_list}")
        print(f" - Rich Tokens : {tokens_rich_fmt}")
        print(f" - Hit Synonyms: {hit_synonyms if hit_synonyms else 'None (Rank boost by other factors)'}")

        print(f"[Fact Check]")
        print(f" - Answer in Doc: {'YES' if has_answer else 'NO'}")
        if has_answer:
            print(f" - Context Snippet: ...{snippet}...")
        else:
            print(f" - Context Snippet: ...{snippet}...")

        print("=" * 80)
        print("\n")

In [ ]:
# 1. 쿼리 분석 결과 가져오기 (이미 하신 거)
query = dataset['train'][123]['question']
tokens_dict = target_retriever._query_tokenizer(query)

# 2. 오리지널 vs 유의어 분리 (가중치로 구분)
# (보통 원본은 1.0 이상, 유의어는 1.0 미만으로 설정하셨을 테니)
original_keywords = {k for k, v in tokens_dict.items() if v >= 1.0}
synonym_keywords = {k for k, v in tokens_dict.items() if v < 1.0}

print(f"🔹 원본 키워드 ({len(original_keywords)}개): {list(original_keywords)[:5]}...")
print(f"🔸 유의어 키워드 ({len(synonym_keywords)}개): {list(synonym_keywords)[:5]}...")
print("=" * 60)

# 3. 검색 결과 하나씩 뜯어보며 유의어 찾기
print("🔍 [유의어 적중 분석 시작]\n")

for i, node_with_score in enumerate(results[:5]): # 상위 5개만
    full_text = node_with_score.node.get_content()
    doc_id = node_with_score.node.metadata.get('docid') or "N/A"

    # 해당 문서에 포함된 단어 찾기
    hit_originals = [w for w in original_keywords if w in full_text]
    hit_synonyms = [w for w in synonym_keywords if w in full_text]

    print(f"📄 [Rank {i+1}] (ID: {doc_id}) Score: {node_with_score.score:.4f}")

    # 🎯 핵심: 유의어가 있으면 강조!
    if hit_synonyms:
        print(f"   🔥 [유의어 적중!] 발견된 유의어: {hit_synonyms}")
    else:
        print(f"   💨 (유의어 없음 - 원본 단어만 매칭됨)")

    print(f"   ✅ [원본 단어 매칭]: {hit_originals}")

    # 문맥 확인 (유의어가 포함된 문장 출력)
    if hit_synonyms:
        print("   👀 [문맥 확인]:")
        for syn in hit_synonyms:
            # 유의어가 포함된 문장 주변 30자만 잘라서 보여주기
            idx = full_text.find(syn)
            start = max(0, idx - 20)
            end = min(len(full_text), idx + 20)
            snippet = full_text[start:end].replace('\n', ' ')
            print(f"      ...{snippet}...")

    print("-" * 60)